In [12]:
import jetson_utils

In [13]:
import requests
#!/usr/bin/env python3
import io
import PIL
import logging
import torch
import torchvision.transforms.functional as F

import numpy as np

from jetson_utils import cudaImage, cudaFromNumpy


ImageTypes = (PIL.Image.Image, np.ndarray, torch.Tensor, cudaImage)
ImageExtensions = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')


def is_image(image):
    """
    Returns true if the object is a PIL.Image, np.ndarray, torch.Tensor, or jetson_utils.cudaImage
    """
    return isinstance(image, ImageTypes)
    
 
def image_size(image):
    """
    Returns the dimensions of the image as a ``(height, width, channels)`` tuple.
    """
    if isinstance(image, (cudaImage, np.ndarray, torch.Tensor)):
        return image.shape
    elif isinstance(image, PIL.Image.Image):
        return image.size
    else:
        raise TypeError(f"expected an image of type {ImageTypes} (was {type(image)})")
        
    
def load_image(path):
    """
    Load an image from a local path or URL that will be downloaded.
    
    Args:
      path (str): either a path or URL to the image.
      
    Returns:
      ``PIL.Image`` instance
    """
    if path.startswith('http') or path.startswith('https'):
        logging.debug(f'-- downloading {path}')
        response = requests.get(path)
        image = PIL.Image.open(io.BytesIO(response.content)).convert('RGB')
    else:
        logging.debug(f'-- loading {path}')
        image = PIL.Image.open(path).convert('RGB')
        
    return image


def cuda_image(image):
    """
    Convert an image from `PIL.Image`, `np.ndarray`, `torch.Tensor`, or `__gpu_array_interface__`
    to a jetson_utils.cudaImage on the GPU (without using memory copies when possible)
    """   
    # TODO implement __gpu_array_interface__
    # TODO torch image formats https://github.com/dusty-nv/jetson-utils/blob/f0bff5c502f9ac6b10aa2912f1324797df94bc2d/python/examples/cuda-from-pytorch.py#L47
    if not is_image(image):
        raise TypeError(f"expected an image of type {ImageTypes} (was {type(image)})")
        
    if isinstance(image, cudaImage):
        return image
        
    if isinstance(image, PIL.Image.Image):
        image = np.asarray(image)  # no copy
        
    if isinstance(image, np.ndarray):
        return cudaFromNumpy(image)
        
    if isinstance(image, torch.Tensor):
        input = input.to(memory_format=torch.channels_last)   # or tensor.permute(0, 3, 2, 1)
        
        return cudaImage(
            ptr=input.data_ptr(), 
            width=input.shape[-1], 
            height=input.shape[-2], 
            format=torch_image_format(input)
        )
        
 
def torch_image(image, dtype=None, device=None):
    """
    Convert the image to a type that is compatible with PyTorch ``(torch.Tensor, ndarray, PIL.Image)``
    """
    if not isinstance(image, ImageTypes):
        raise TypeError(f"expected an image of type {ImageTypes} (was {type(image)})")
        
    if isinstance(image, cudaImage):
        return torch.as_tensor(image, dtype=dtype, device=device)
    elif isinstance(image, (PIL.Image.Image, np.ndarray)):
        image = F.to_tensor(image)

    return image.to(dtype=dtype, device=device)
        
        
def torch_image_format(tensor):
    """
    Determine the cudaImage format string (eg 'rgb32f', 'rgba32f', ect) from a PyTorch tensor.
    Only float and uint8 tensors are supported because those datatypes are supported by cudaImage.
    """
    if tensor.dtype != torch.float32 and tensor.dtype != torch.uint8:
        raise ValueError(f"PyTorch tensor datatype should be torch.float32 or torch.uint8 (was {tensor.dtype})")
        
    if len(tensor.shape)>= 4:     # NCHW layout
        channels = tensor.shape[1]
    elif len(tensor.shape) == 3:   # CHW layout
        channels = tensor.shape[0]
    elif len(tensor.shape) == 2:   # HW layout
        channels = 1
    else:
        raise ValueError(f"PyTorch tensor should have at least 2 image dimensions (has {tensor.shape.length})")
        
    if channels == 1:   return 'gray32f' if tensor.dtype == torch.float32 else 'gray8'
    elif channels == 3: return 'rgb32f'  if tensor.dtype == torch.float32 else 'rgb8'
    elif channels == 4: return 'rgba32f' if tensor.dtype == torch.float32 else 'rgba8'
    
    raise ValueError(f"PyTorch tensor should have 1, 3, or 4 image channels (has {channels})")

In [14]:
#!/usr/bin/env python3
import time
import queue
import threading
import logging
import traceback


class Plugin(threading.Thread):
    """
    Base class for plugins that process incoming/outgoing data from connections
    with other plugins, forming a pipeline or graph.  Plugins can run either
    single-threaded or in an independent thread that processes data out of a queue.

    Frequent categories of plugins:
    
      * sources:  text prompts, images/video
      * process:  LLM queries, RAG, dynamic LLM calls, image post-processors
      * outputs:  print to stdout, save images/video
      
    Inherited plugins should implement the :func:`process` function to handle incoming data.
    
    Args:
    
      output_channels (int): the number of sets of output connections the plugin has
      relay (bool): if true, will relay any inputs as outputs after processing
      drop_inputs (bool): if true, only the most recent input in the queue will be used
      threaded (bool): if true, will spawn independent thread for processing the queue.
    """
    def __init__(self, output_channels=1, relay=False, drop_inputs=False, threaded=True, **kwargs):
        """
        Initialize plugin
        """
        super().__init__(daemon=True)

        self.relay = relay
        self.drop_inputs = drop_inputs
        self.threaded = threaded
        self.interrupted = False
        self.processing = False
        
        self.outputs = [[] for i in range(output_channels)]
        self.output_channels = output_channels
        
        if threaded:
            self.input_queue = queue.Queue()
            self.input_event = threading.Event()

    def process(self, input, **kwargs):
        """
        Abstract function that plugin instances should implement to process incoming data.
        Don't call this function externally unless ``threaded=False``, because
        otherwise the plugin's internal thread dispatches from the queue.

        Args:
        
          input: input data to process from the previous plugin in the pipeline
          kwargs: optional processing arguments that accompany this data
          
        Returns:
        
          Plugins should return their output data to be sent to downstream plugins.
          You can also call :func:`output()` as opposed to returning it.
        """
        raise NotImplementedError(f"plugin {type(self)} has not implemented process()")
    
    def add(self, plugin, channel=0, **kwargs):
        """
        Connect the output queue from this plugin with the input queue of another plugin,
        so that this plugin sends its output data to the other one.
        
        Args:
        
          plugin (Plugin|callable): either the plugin to link to, or a callback function.
          channel (int) -- the output channel of this plugin to link the other plugin to.
                        
        Returns:

          A reference to this plugin instance (self)
        """
        class Callback(Plugin):
            """
            Wrapper for calling a function with the same signature as Plugin.process()
            This is automatically used by Plugin.add() so it's typically not needed.
            Callbacks are threaded by default and will be run asynchronously.
            If it's a lightweight non-blocking function, you can set threaded=False
            """
            def __init__(self, function, threaded=False, **kwargs):
                """
                Parameters:
                  function (callable) -- function for processing data like Plugin.process() would
                """
                super().__init__(threaded=threaded, **kwargs)
                self.function = function
                
            def process(self, input, **kwargs):
                return self.function(input, **kwargs)
        
        if not isinstance(plugin, Plugin):
            if not callable(plugin):
                raise TypeError(f"{type(self)}.add() expects either a Plugin instance or a callable function (was {type(plugin)})")
            plugin = Callback(plugin, **kwargs)
            
        self.outputs[channel].append(plugin)
        
        if isinstance(plugin, Callback):
            logging.debug(f"connected {type(self).__name__} to {plugin.function.__name__} on channel={channel}")  # TODO https://stackoverflow.com/a/25959545
        else:
            logging.debug(f"connected {type(self).__name__} to {type(plugin).__name__} on channel={channel}")
            
        return self
    
    def __call__(self, input=None, **kwargs):
        """
        Callable () operator alias for the :func:`input()` function.
        This is provided for a more intuitive way of processing data 
        like ``plugin(data)`` instead of ``plugin.input(data)``
        
        Args:
        
          input: input data sent to the plugin's :func:`process()` function.
          kwargs: additional arguments forwarded to the plugin's :func:`process()` function.
          
        Returns:
        
          None if the plugin is threaded, otherwise returns any outputs.
        """
        return self.input(input, **kwargs)
        
    def input(self, input=None, **kwargs):
        """
        Add data to the plugin's processing queue and return immediately,
        or process it now and return the results if ``threaded=False``.
        
        Args:
        
          input: input data sent to the plugin's :func:`process()` function.
          kwargs: additional arguments forwarded to the plugin's :func:`process()` function.
          
        Returns:
        
          None if the plugin is threaded, otherwise returns any outputs.
        """
        if self.threaded:
            #self.start() # thread may not be started if plugin only called from a callback
            if self.drop_inputs:
                configs = []
                while True:
                    try:
                        config_input, config_kwargs = self.input_queue.get(block=False)
                        if config_input is None and len(config_kwargs) > 0:  # still apply config
                            configs.append((config_input, config_kwargs))
                    except queue.Empty:
                        break
                for config in configs:
                    self.input_queue.put(config)
                    self.input_event.set()

            self.input_queue.put((input,kwargs))
            self.input_event.set()
        else:
            self.dispatch(input, **kwargs)
            
    def output(self, output, channel=0, **kwargs):
        """
        Output data to the next plugin(s) on the specified channel (-1 for all channels)
        """
        if output is None:
            return
            
        if channel >= 0:
            for output_plugin in self.outputs[channel]:
                output_plugin.input(output, **kwargs)
        else:
            for output_channel in self.outputs:
                for output_plugin in output_channel:
                    output_plugin.input(output, **kwargs)
                    
        return output
     
    @property
    def num_outputs(self):
        """
        Return the total number of output connections across all channels
        """
        count = 0
        for output_channel in self.outputs:
            count += len(output_channel) 
        return count
        
    def start(self):
        """
        Start threads for all plugins in the graph that have threading enabled.
        """
        if self.threaded:
            if not self.is_alive():
                super().start()
            
        for output_channel in self.outputs:
            for output in output_channel:
                output.start()
                
        return self
            
    def run(self):
        """
        Processes the queue forever and automatically run when created with ``threaded=True``
        """
        while True:
            try:
                self.input_event.wait()
                self.input_event.clear()
                
                while True:
                    try:
                        input, kwargs = self.input_queue.get(block=False)
                        self.dispatch(input, **kwargs)
                    except queue.Empty:
                        break
            except Exception as error:
                logging.error(f"Exception occurred during processing of {type(self)}\n\n{traceback.format_exc()}")

    def dispatch(self, input, **kwargs):
        """
        Invoke the process() function on incoming data
        """
        if self.interrupted:
            #logging.debug(f"{type(self)} resetting interrupted flag to false")
            self.interrupted = False
          
        self.processing = True
        outputs = self.process(input, **kwargs)
        self.processing = False

        self.output(outputs)
        
        if self.relay:
            self.output(input)
            
        return outputs
   
    def interrupt(self, clear_inputs=True, recursive=True, block=None):
        """
        Interrupt any ongoing/pending processing, and optionally clear the input queue
        along with any downstream queues, and optionally wait for processing of those 
        requests to have finished.
        
        Args:
        
          clear_inputs (bool):  if True, clear any remaining inputs in this plugin's queue.
          recursive (bool):  if True, then any downstream plugins will also be interrupted.
          block (bool):  is true, this function will wait until any ongoing processing has finished.
                         This is done so that any lingering outputs don't cascade downstream in the pipeline.
                         If block is None, it will automatically be set to true if this plugin has outputs.
        """
        #logging.debug(f"interrupting plugin {type(self)}  clear_inputs={clear_inputs} recursive={recursive} block={block}")
        
        if clear_inputs:
            self.clear_inputs()
          
        self.interrupted = True
        
        num_outputs = self.num_outputs
        block_other = block
        
        if block is None and num_outputs > 0:
            block = True
            
        while block and self.processing:
            #logging.debug(f"interrupt waiting for {type(self)} to complete processing")
            time.sleep(0.01) # TODO use an event for this?
        
        if recursive and num_outputs > 0:
            for output_channel in self.outputs:
                for output in output_channel:
                    output.interrupt(clear_inputs=clear_inputs, recursive=recursive, block=block_other)
                    
    def clear_inputs(self):
        """
        Clear the input queue, dropping any data.
        """
        while True:
            try:
                self.input_queue.get(block=False)
            except queue.Empty:
                return         

    def find(self, type):
        """
        Return the plugin with the specified type by searching for it among
        the pipeline graph of inputs and output connections to other plugins.
        """
        if isinstance(self, type):
            return self
            
        for output_channel in self.outputs:
            for output in output_channel:
                if isinstance(output, type):
                    return output
                plugin = output.find(type)
                if plugin is not None:
                    return plugin
            
        return None
    
    '''
    def __getitem__(self, type):
        """
        Subscript indexing [] operator alias for find()
        """
        return self.find(type)
    '''       

In [15]:
import time
import logging
import traceback

import torch
import numpy as np


from jetson_utils import videoSource, videoOutput, cudaDeviceSynchronize, cudaToNumpy


class VideoSource(Plugin):
    """
    Captures or loads a video/camera stream or sequence of images
    https://github.com/dusty-nv/jetson-inference/blob/master/docs/aux-streaming.md
    """
    def __init__(self, video_input='/dev/video0', 
                 video_input_width=None, video_input_height=None, 
                 video_input_codec=None, video_input_framerate=None, 
                 video_input_save=None, return_tensors='cuda', **kwargs):
        """
        Parameters:
        
          input (str) -- path to video file, directory of images, or stream URL
          input_width (int) -- the disired width in pixels (default uses stream's resolution)
          input_height (int) -- the disired height in pixels (default uses stream's resolution)
          input_codec (str) -- force a particular codec ('h264', 'h265', 'vp8', 'vp9', 'mjpeg', ect)
          return_tensors (str) -- the object datatype of the image to output ('np', 'pt', 'cuda')
        """
        super().__init__(**kwargs)
        
        options = {}
        
        if video_input_width:
            options['width'] = video_input_width
            
        if video_input_height:
            options['height'] = video_input_height
            
        if video_input_codec:
            options['codec'] = video_input_codec
 
        if video_input_framerate:
            options['framerate'] = video_input_framerate
            
        if video_input_save:
            options['save'] = video_input_save
        
        self.stream = videoSource(video_input, options=options)
        self.file = (self.stream.GetOptions()['resource']['protocol'] == 'file')
        self.options = options
        self.resource = video_input  # self.stream.GetOptions().resource['string']
        self.return_tensors = return_tensors

    def capture(self, timeout=2500, retries=8, return_tensors=None):
        """
        Capture images from the video source as long as it's streaming
        """
        if not return_tensors:
            return_tensors = self.return_tensors
            
        retry = 0
        
        while retry < retries:
            image = self.stream.Capture(format='rgb8', timeout=timeout)

            if image is None:
                if self.file:
                    break
                logging.warning(f"video source {self.resource} timed out during capture, re-trying...")
                retry = retry + 1
                continue
   
            if return_tensors == 'pt':
                image = torch.as_tensor(image, device='cuda')
            elif return_tensors == 'np':
                image = cudaToNumpy(image)
                cudaDeviceSynchronize()
            elif return_tensors != 'cuda':
                raise ValueError(f"return_tensors should be 'np', 'pt', or 'cuda' (was '{return_tensors}')")
                
            self.output(image)
            return image
    
        return None
        
    def reconnect(self):
        """
        Attempt to re-open the stream if the connection fails
        """
        while True:
            try:
                if self.stream is not None:
                    self.stream.Close()
                    self.stream = None        
            except Exception as error:
                logging.error(f"Exception occurred closing video source \"{self.resource}\"\n\n{traceback.format_exc()}")

            try:
                self.stream = videoSource(self.resource, options=self.options)
                return
            except Exception as error:
                logging.error(f"Failed to create video source \"{self.resource}\"\n\n{traceback.format_exc()}")
                time.sleep(2.5)
            
    def run(self):
        """
        Run capture continuously and attempt to handle disconnections
        """
        while True:
            try:
                img = self.capture()
            except Exception as error:
                logging.error(f"Exception occurred during video source capture of \"{self.resource}\"\n\n{traceback.format_exc()}")
            
            if img is None:
                if self.file:
                    return
                logging.error(f"Re-initializing video source \"{self.resource}\"")
                self.reconnect()

    @property
    def streaming(self):
        """
        Returns true if the stream is currently open, false if closed or EOS.
        """
        return self.stream.IsStreaming()
     
    @property
    def eos(self):
        """
        Returns true if the stream is currently closed (EOS has been reached)
        """
        return not self.streaming
        
class VideoOutput(Plugin):
    """
    Saves images to a compressed video or directory of individual images, the display, or a network stream.
    https://github.com/dusty-nv/jetson-inference/blob/master/docs/aux-streaming.md
    """
    def __init__(self, video_output=None, video_output_codec=None, video_output_bitrate=None, video_output_save=None, **kwargs):
        """
        Parameters:
        
          input (str) -- path to video file, directory of images, or stream URL
          output_codec (str) -- force a particular codec ('h264', 'h265', 'vp8', 'vp9', 'mjpeg', ect)
          output_bitrate (int) -- the desired bitrate in bits per second (default is 4 Mbps)
        """
        super().__init__(**kwargs)
        
        options = {}

        if video_output_codec:
            options['codec'] = video_output_codec
            
        if video_output_bitrate:
            options['bitrate'] = video_output_bitrate

        if video_output_save:
            options['save'] = video_output_save
            
        if video_output is None:
            video_output = ''
            
        args = None if 'display://' in video_output else ['--headless']
        
        self.stream = videoOutput(video_output, options=options, argv=args)
        self.resource = video_output
        
    def process(self, input, **kwargs):
        """
        Input should be a jetson_utils.cudaImage, np.ndarray, torch.Tensor, or have __cuda_array_interface__
        """
        self.stream.Render(cuda_image(input))

In [20]:
def on_video(image):
    num_frames = video_source.stream.GetFrameCount()
    if num_frames % 25 == 0:
        logging.info(f'captured {num_frames} frames ({image.width}x{image.height}) from {video_source.resource}')
        
video_source = VideoSource(video_input='file:///home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4')

video_source.add(on_video, threaded=False)
videoOutput.Render()

video_source.start().join()

[gstreamer] gstDecoder -- creating decoder for /home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4
Opening in BLOCKING MODE 
[gstreamer] gstDecoder -- discovered video resolution: 960x540  (framerate 30.000000 Hz)
[gstreamer] gstDecoder -- discovered video caps:  video/x-h264, stream-format=(string)byte-stream, alignment=(string)au, level=(string)3.1, profile=(string)high, width=(int)960, height=(int)540, framerate=(fraction)30/1, pixel-aspect-ratio=(fraction)1/1, chroma-format=(string)4:2:0, bit-depth-luma=(uint)8, bit-depth-chroma=(uint)8, parsed=(boolean)true
[gstreamer] gstDecoder -- pipeline string:
[gstreamer] filesrc location=/home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4 ! qtdemux ! queue ! h264parse ! nvv4l2decoder name=decoder enable-max-performance=1 ! video/x-raw(memory:NVMM) ! nvvidconv name=vidconv ! video/x-raw ! appsink name=mysink
[video]  created gstDecoder from file:///home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4
----

NvMMLiteOpen : Block : BlockType = 261 
NvMMLiteBlockCreate : Block : BlockType = 261 

(python:6921): GStreamer-CRITICAL **: 16:54:21.369: gst_debug_log_valist: assertion 'category != NULL' failed

(python:6921): GStreamer-CRITICAL **: 16:54:21.369: gst_debug_log_valist: assertion 'category != NULL' failed

(python:6921): GStreamer-CRITICAL **: 16:54:21.369: gst_debug_log_valist: assertion 'category != NULL' failed

(python:6921): GStreamer-CRITICAL **: 16:54:21.369: gst_debug_log_valist: assertion 'category != NULL' failed


[gstreamer] gstDecoder -- pipeline stopped
[gstreamer] opening gstDecoder for streaming, transitioning pipeline to GST_STATE_PLAYING
Opening in BLOCKING MODE 
[gstreamer] gstreamer changed state from NULL to READY ==> mysink
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter10
[gstreamer] gstreamer changed state from NULL to READY ==> vidconv
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter9
[gstreamer] gstreamer changed state from NULL to READY ==> decoder
[gstreamer] gstreamer changed state from NULL to READY ==> h264parse1
[gstreamer] gstreamer changed state from NULL to READY ==> queue2
[gstreamer] gstreamer changed state from NULL to READY ==> qtdemux3
[gstreamer] gstreamer changed state from NULL to READY ==> filesrc2
[gstreamer] gstreamer changed state from NULL to READY ==> pipeline3
[gstreamer] gstreamer changed state from READY to PAUSED ==> capsfilter10
[gstreamer] gstreamer changed state from READY to PAUSED ==> vidconv
[gstreamer] gs

NvMMLiteOpen : Block : BlockType = 261 
NvMMLiteBlockCreate : Block : BlockType = 261 


[cuda]   allocated 4 ring buffers (777607 bytes each, 3110428 bytes total)
[cuda]   allocated 4 ring buffers (8 bytes each, 32 bytes total)
[gstreamer] gstreamer changed state from READY to PAUSED ==> mysink
[gstreamer] gstreamer changed state from READY to PAUSED ==> pipeline3
[gstreamer] gstreamer message async-done ==> pipeline3
[gstreamer] gstreamer message latency ==> mysink
[gstreamer] gstreamer message new-clock ==> pipeline3
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> mysink
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> capsfilter10
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> vidconv
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> capsfilter9
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> decoder
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> h264parse1
[gstreamer] gstreamer changed state from PAUSED to PLAYING ==> queue2
[gstreamer] gstreamer changed state from PAUSED to PLAYI

In [22]:
!export DISPLAY=:0
!xinit &
import logging

# Asumimos que ya tienes importadas las clases necesarias, como videoSource y videoOutput
from jetson_utils import videoSource, videoOutput

def main():
    # Configura la fuente del video y el output
    video_input = 'file:///home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4'
    video_output = 'display://0'

    # Crea las instancias de VideoSource y VideoOutput
    source = videoSource(video_input)  # Asume que hay parámetros predeterminados adecuados
    output = videoOutput(video_output)  # Asume que hay parámetros predeterminados adecuados

    # Bucle para capturar y renderizar el vídeo frame por frame
    try:
        while source.IsStreaming():
            frame = source.Capture()  # Captura un frame
            if frame is not None:
                output.Render(frame)  # Renderiza el frame
            else:
                break  # Sale del bucle si no hay más frames
    except KeyboardInterrupt:
        print("Interrupción por parte del usuario.")
    except Exception as e:
        logging.error(f"Ocurrió un error durante la captura y renderización del video: {str(e)}")

    # Limpieza: Asegúrate de cerrar adecuadamente los recursos
    source.Close()
    output.Close()

if __name__ == "__main__":
    main()

[gstreamer] gstDecoder -- creating decoder for /home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4
Opening in BLOCKING MODE 
[gstreamer] gstDecoder -- discovered video resolution: 960x540  (framerate 30.000000 Hz)
[gstreamer] gstDecoder -- discovered video caps:  video/x-h264, stream-format=(string)byte-stream, alignment=(string)au, level=(string)3.1, profile=(string)high, width=(int)960, height=(int)540, framerate=(fraction)30/1, pixel-aspect-ratio=(fraction)1/1, chroma-format=(string)4:2:0, bit-depth-luma=(uint)8, bit-depth-chroma=(uint)8, parsed=(boolean)true
[gstreamer] gstDecoder -- pipeline string:
[gstreamer] filesrc location=/home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4 ! qtdemux ! queue ! h264parse ! nvv4l2decoder name=decoder enable-max-performance=1 ! video/x-raw(memory:NVMM) ! nvvidconv name=vidconv ! video/x-raw ! appsink name=mysink
[video]  created gstDecoder from file:///home/johnny/Projects/small-fast-detector/misc/repaired_file.mp4
----

NvMMLiteOpen : Block : BlockType = 261 
NvMMLiteBlockCreate : Block : BlockType = 261 


[OpenGL] glDisplay -- display device initialized (1920x1080)
[video]  created glDisplay from display://0
------------------------------------------------
glDisplay video options:
------------------------------------------------
  -- URI: display://0
     - protocol:  display
     - location:  0
  -- deviceType: display
  -- ioType:     output
  -- width:      1920
  -- height:     1080
  -- frameRate:  0
  -- numBuffers: 4
  -- zeroCopy:   true
------------------------------------------------
[gstreamer] opening gstDecoder for streaming, transitioning pipeline to GST_STATE_PLAYING
Opening in BLOCKING MODE 
[gstreamer] gstreamer changed state from NULL to READY ==> mysink
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter16
[gstreamer] gstreamer changed state from NULL to READY ==> vidconv
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter15
[gstreamer] gstreamer changed state from NULL to READY ==> decoder
[gstreamer] gstreamer changed state from N

NvMMLiteOpen : Block : BlockType = 261 
NvMMLiteBlockCreate : Block : BlockType = 261 


[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bitrate=(uint)400320;
[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bitrate=(uint)314880;
[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bitrate=(uint)266160;
[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bitrate=(uint)244560;
[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bitrate=(uint)225600;
[gstreamer] gstreamer mysink taglist, video-codec=(string)"H.264\ \(High\ Profile\)", maximum-bitrate=(uint)1357879, bitrate=(uint)1357879, minimum-bi